In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# 数据加载
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [3]:
# 数据探索
# 查看train_data信息
#pd.set_option('display.max_columns', None) #显示所有列
print('查看数据信息：列名、非空个数、类型等')
print(train_data.info())
print('-'*30)
print('查看数据摘要')
print(train_data.describe())
print('-'*30)
print('查看离散数据分布')
print(train_data.describe(include=['O']))
print('-'*30)
print('查看前5条数据')
print(train_data.head())
print('-'*30)
print('查看后5条数据')
print(train_data.tail())

查看数据信息：列名、非空个数、类型等
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
------------------------------
查看数据摘要
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642

In [4]:
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
# 使用票价的均值填充票价中的nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

print(train_data['Embarked'].value_counts())
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]
print('特征值')
print(train_features)

S    644
C    168
Q     77
Name: Embarked, dtype: int64
特征值
     Pclass     Sex        Age  SibSp  Parch     Fare Embarked
0         3    male  22.000000      1      0   7.2500        S
1         1  female  38.000000      1      0  71.2833        C
2         3  female  26.000000      0      0   7.9250        S
3         1  female  35.000000      1      0  53.1000        S
4         3    male  35.000000      0      0   8.0500        S
..      ...     ...        ...    ...    ...      ...      ...
886       2    male  27.000000      0      0  13.0000        S
887       1  female  19.000000      0      0  30.0000        S
888       3  female  29.699118      1      2  23.4500        S
889       1    male  26.000000      0      0  30.0000        C
890       3    male  32.000000      0      0   7.7500        Q

[891 rows x 7 columns]


In [5]:
dvec=DictVectorizer(sparse=False)
train_features=dvec.fit_transform(train_features.to_dict(orient='record'))
test_features=dvec.transform(test_features.to_dict(orient='record'))
print(dvec.feature_names_)

['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


In [6]:
train_ss_x = train_features
test_ss_x = test_features
train_y = train_labels

In [7]:
# 创建LR分类器
lr = LogisticRegression(solver='liblinear', multi_class='auto') #数据集比较小，使用liblinear，数据集大使用 sag或者saga
lr.fit(train_ss_x, train_y)
predict_y=lr.predict(test_ss_x)
acc_lr = round(lr.score(train_features, train_labels), 6)
print(u'LR算法score准确率为 %.4lf' % acc_lr)
print(u'LR算法cross_val_score准确率为 %.4lf\n' % np.mean(cross_val_score(lr, train_features, train_labels, cv=10)))

LR算法score准确率为 0.8036
LR算法cross_val_score准确率为 0.7946



In [8]:
# 创建贝叶斯分类器
gnb = GaussianNB()
gnb.fit(train_ss_x,train_y)
predict_y=gnb.predict(test_ss_x)
acc_gnb = round(gnb.score(train_features, train_labels), 6)
print(u'贝叶斯算法score准确率为 %.4lf' % acc_gnb)
print(u'贝叶斯算法cross_val_score准确率为 %.4lf\n' % np.mean(cross_val_score(gnb, train_features, train_labels, cv=10)))

贝叶斯算法score准确率为 0.7912
贝叶斯算法cross_val_score准确率为 0.7845



In [9]:
# 创建SVM分类器
model = svm.SVC(kernel='rbf', C=1.0, gamma='auto')
model.fit(train_ss_x,train_y)
predict_y=model.predict(test_ss_x)
acc_svm = round(model.score(train_features, train_labels), 6)
print(u'SVM算法score准确率为 %.4lf' % acc_svm)
print(u'SVM算法cross_val_score准确率为 %.4lf\n' % np.mean(cross_val_score(model, train_features, train_labels, cv=10)))


SVM算法score准确率为 0.8900
SVM算法cross_val_score准确率为 0.7285



In [10]:
# 创建KNN分类器
knn = KNeighborsClassifier()
knn.fit(train_ss_x,train_y)
predict_y=knn.predict(test_ss_x)
acc_knn = round(knn.score(train_features, train_labels), 6)
print(u'KNN算法score准确率为 %.4lf' % acc_knn)
print(u'KNN算法cross_val_score准确率为 %.4lf\n' % np.mean(cross_val_score(knn, train_features, train_labels, cv=10)))

KNN算法score准确率为 0.8193
KNN算法cross_val_score准确率为 0.7094

